In [4]:
from rqcopt import brickwall_unitary_hessian_matrix, brickwall_unitary_gradient_vector, brickwall_unitary
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt


# Parameters for the Ising Hamiltonian
# L has to be even! Due to K only being able to control even Ls!
L = 4

# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.IsingHamiltonian(field, 1, 0, 0).as_matrix().toarray()
#hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
perms = [[i for i in range(L)], [i for i in range(1, L)]+[0]]

eigenvalues, eigenvectors = LA.eig(hamil)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]
print("Ground State Energy", -eigenvalues_sort[-1].real)

Ground State Energy -4.0


In [7]:
def construct_ising_local_term(J, g):
    """
    Construct local interaction term of Ising Hamiltonian on a one-dimensional
    lattice for interaction parameter `J` and external field parameter `g`.
    """
    # Pauli-X and Z matrices
    X = np.array([[0.,  1.], [1.,  0.]])
    Z = np.array([[1.,  0.], [0., -1.]])
    I = np.identity(2)
    return J*np.kron(Z, Z) + g*0.5*(np.kron(X, I) + np.kron(I, X))

In [13]:
import scipy
U = scipy.linalg.expm(-1j * hamil)
hloc = construct_ising_local_term(1, 0)
Vlist = [scipy.linalg.expm(-1j*hloc) for c in [0, 1]]
np.linalg.norm(U - brickwall_unitary(Vlist, L, perms), ord=2)

2.482534153247273e-16

In [14]:
np.linalg.norm(brickwall_unitary_gradient_vector(Vlist, L, U, perms), ord=2)

1.7316902782981841e-15

In [17]:
hess = brickwall_unitary_hessian_matrix(Vlist, L, U, perms)

In [20]:
np.all(np.linalg.eigvals(hess) >= 1e-1)

False